# OBJETIVO

-- EXTRAIR AS BASES CADASTRAIS POR HOSPITAL DIARIAMENTE DO QVIS, AUTOMÁTICO COM SELENIUM

-- MOVER AS BASES PARA A PASTA CORRETA DO CADASTRO SOLUCX

# IMPORTS

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
import os
# from datetime import datetime
import shutil
from send2trash import send2trash
import math
import pandas as pd

In [10]:
from datetime import datetime
inicio_tempo = datetime.now()

# LIMPAR ARQUIVOS QUE ESTEJAM NOMEADOS COMO DATA NOS DOWNLOADS

In [11]:
# Define o caminho da pasta Downloads
# downloads_folder = r"caminho downloads Gaby"
downloads_folder = r"C:\Users\MarceloFridmanMarizd\Downloads"

# Itera sobre os arquivos na pasta
for file in os.listdir(downloads_folder):
    file_path = os.path.join(downloads_folder, file)
    
    # Verifica se o arquivo é um .xlsx e começa com "data"
    if file.startswith("data") and file.endswith(".xlsx") and os.path.isfile(file_path):
        send2trash(file_path)  # Move para a Lixeira
        print(f"Moved to Trash: {file_path}")

print("Cleanup complete.")

Cleanup complete.


# EXTRAÇÃO DAS BASES CADASTRAIS POR HOSPITAL NO BI CADASTRO QVIS

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    ElementClickInterceptedException,
    ElementNotInteractableException,
    TimeoutException
)

import time
import os
from datetime import datetime


url = "https://app.powerbi.com/groups/me/reports/d86aa107-fd90-4155-9500-7d4ee4d6e138/ReportSection59a071425900c5b582a8?experience=power-bi"

def reset_filters(driver):
    try:
        reset_icon = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "mat-icon[fonticon='pbi-glyph-reset']"))
        )
        reset_icon.click()
        print("\U0001F539 Resetando filtros...")
        time.sleep(30)

        redefinir_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.ID, "okButton"))
        )
        redefinir_button.click()
        print("\U0001F539 Filtros resetados.")
        time.sleep(30)
    except Exception as e:
        print(f"\u26A0\uFE0F Erro ao resetar filtros: {e}")

def export_data(driver):
    try:
        admissao_column_header = WebDriverWait(driver, 45).until(
            EC.presence_of_element_located((By.XPATH, "//div[@role='columnheader' and contains(text(), 'Cód. Admissão')]"))
        )

        # Primeiro clique
        admissao_column_header.click()
        print("Clicando na coluna Cód. Admissão e esperando 60 segundos...")
        time.sleep(60)

        # Segundo clique
        admissao_column_header = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, "//div[@role='columnheader' and contains(text(), 'Cód. Admissão')]"))
        )
        admissao_column_header.click()
        print("Clicando novamente na coluna Cód. Admissão e esperando mais 60 segundos...")
        time.sleep(60)

        # Clique adicional no novo XPath
        extra_click_element = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='pvExplorationHost']/div/div/exploration/div/explore-canvas/div/div[2]/div/div[2]/div[2]/visual-container-repeat/visual-container[28]/transform/div/div[3]/div/div/visual-modern/div/div/div[2]/div[1]/div[2]"))
        )
        extra_click_element.click()
        print("Clicando no elemento adicional e aguardando 30 segundos...")
        time.sleep(30)

        from selenium.webdriver import ActionChains

        # Hover no visual que contém o botão
        # Hover no visual que contém o botão "Mais opções"
        visual_container = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, "//visual-container[.//button[@aria-label='Mais opções']]"))
        )
        ActionChains(driver).move_to_element(visual_container).perform()
        
        # Espera botão "Mais opções" ficar clicável
        mais_opcoes_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label='Mais opções']"))
        )
        
        # Clica no "Mais opções" (com fallback JS se falhar)
        try:
            mais_opcoes_button.click()
        except:
            driver.execute_script("arguments[0].click();", mais_opcoes_button)
        
        # Agora espera o menu ficar visível e clica no "Exportar dados"
        exportar_dados_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='pbimenu-item.Exportar dados']"))
        )
        try:
            exportar_dados_button.click()
        except:
            driver.execute_script("arguments[0].click();", exportar_dados_button)
        
        # Espera o modal de exportação e clica no "Exportar"
        exportar_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label='Exportar']"))
        )
        exportar_button.click()
        print("✅ Exportação iniciada.")

        
        time.sleep(30)
        rename_downloaded_file()
        time.sleep(30)
    
    except Exception as e:
        print(f"\u26A0\uFE0F Erro na exportação: {e}")

def rename_downloaded_file():
    download_folder = r"C:\Users\MarceloFridmanMarizd\Downloads"
    current_date = datetime.now().strftime("%Y%m")

    print("\U0001F4C2 Aguardando o download do arquivo...")

    # Tempo máximo de espera pelo download (60s)
    timeout = 120
    start_time = time.time()
    
    while True:
        files_in_folder = os.listdir(download_folder)
        excel_files = [f for f in files_in_folder if "data" in f.lower() and f.endswith(".xlsx")]
        
        if excel_files:
            file_name = excel_files[0]  # Pega o primeiro arquivo encontrado
            break
        
        if time.time() - start_time > timeout:
            print("\u26A0\uFE0F Tempo limite atingido. Nenhum arquivo encontrado para renomear.")
            return
        
        time.sleep(30)  # Espera 30 segundos antes de checar novamente

    base_name = f"Cadastro_{current_date}"
    new_name = f"{base_name}.xlsx"
    old_file_path = os.path.join(download_folder, file_name)
    new_file_path = os.path.join(download_folder, new_name)

    counter = 1
    while os.path.exists(new_file_path):
        new_name = f"{base_name}_{counter}.xlsx"
        new_file_path = os.path.join(download_folder, new_name)
        counter += 1

    os.rename(old_file_path, new_file_path)
    print(f"\U0001F4C2 Arquivo salvo como {new_name}")

print("\U0001F4BB Abrindo o navegador...")
driver = webdriver.Chrome()
driver.get(url)
time.sleep(30)

print("\U0001F4DD Preenchendo o e-mail...")
email_input = driver.find_element(By.ID, "email")
email_input.click()
email_input.send_keys("marketing.bi@visionone.com.br")

submit_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, "submitBtn")))
submit_button.click()
time.sleep(30)

print("\U0001F511 Preenchendo a senha...")
password_input = driver.find_element(By.NAME, "passwd")
password_input.send_keys("MkT_B1i@")
password_input.send_keys(Keys.RETURN)

time.sleep(45)
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException, ElementNotInteractableException, TimeoutException
import time

submit_button = driver.find_element(By.ID, "idSIButton9")

try:
    submit_button.click()
except (ElementClickInterceptedException, ElementNotInteractableException) as e:
    print("Click failed, trying Enter key...")
    # Option 1: send Enter to the button itself
    try:
        submit_button.send_keys(Keys.ENTER)
    except:
        # Option 2: send Enter to the <body> if button isn't focusable
        body = driver.find_element(By.TAG_NAME, "body")
        body.send_keys(Keys.ENTER)



#password_input.send_keys(Keys.RETURN)
#submit_button.click()
time.sleep(60)

print("\U0001F510 Login concluído com sucesso!")

reset_filters(driver)
time.sleep(20)
export_data(driver)

print("\u2705 Processo concluído.")
driver.quit()

💻 Abrindo o navegador...
📝 Preenchendo o e-mail...
🔑 Preenchendo a senha...
🔐 Login concluído com sucesso!
🔹 Resetando filtros...
🔹 Filtros resetados.
Clicando na coluna Cód. Admissão e esperando 60 segundos...
Clicando novamente na coluna Cód. Admissão e esperando mais 60 segundos...
Clicando no elemento adicional e aguardando 30 segundos...
✅ Exportação iniciada.
📂 Aguardando o download do arquivo...
📂 Arquivo salvo como Cadastro_202509.xlsx
✅ Processo concluído.


# MOVER ARQUIVOS DO DOWNLOADS PARA PASTA DE CADASTRO SOLUCX

# SUBSTITUINDO TODOS OS DIAS PARA NÃO ACUMULAR

In [13]:
# Obter a data atual para o nome do arquivo
current_date = datetime.now().strftime("%Y%m")

# Diretório de downloads
# download_folder = r"C:\Users\HenriquedaLuzPacheco\Downloads"  # colocar caminho gaby downloads
download_folder = r"C:\Users\MarceloFridmanMarizd\Downloads"  # Diretório de downloads

# Caminho da pasta de destino (você pode definir conforme necessário)
# destination_folder = r"" colocar caminho gaby pasta cadastro solucx
destination_folder = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\Cadastro"  # Pasta onde os arquivos serão movidos

# Procurar os arquivos baixados com o formato "Cadastro_{data}"
files_in_folder = os.listdir(download_folder)

for file_name in files_in_folder:
    # Verifica se o nome do arquivo está no formato esperado
    if file_name.lower().startswith("cadastro_") and file_name.endswith(f"_{current_date}.xlsx"):
        # Caminho completo do arquivo original e o destino final
        old_file_path = os.path.join(download_folder, file_name)
        new_file_path = os.path.join(destination_folder, file_name)

        # Criar a pasta de destino caso não exista
        os.makedirs(destination_folder, exist_ok=True)

        # Mover o arquivo para a pasta apropriada
        shutil.move(old_file_path, new_file_path)
        print(f"📂 Arquivo {file_name} movido para {destination_folder}")

📂 Arquivo Cadastro_202509.xlsx movido para C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\Cadastro


# CONSOLIDAR OS ARQUIVOS EXCEL EM UM ÚNICO, TODOS OS DIAS, PARA EVITAR LENTIDÃO NA LEITURA

# E EXCLUINDO OS OUTROS ARQUIVOS EXCEL

In [ ]:
import os
import time
import math
import pandas as pd
from datetime import datetime
from send2trash import send2trash
import csv

# Pasta onde os arquivos estão localizados
base_folder = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\Cadastro"

# Lista das colunas que serão utilizadas
colunas_desejadas = ['Cód. Paciente', 'Paciente', 'Gênero', 'Nascimento']

def load_and_save_cadastro_data():
    print(f"Processando arquivos da pasta: {base_folder}")
    time.sleep(10)  # Adicionando delay

    dfs = []
    
    try:
        for file in os.listdir(base_folder):
            file_path = os.path.join(base_folder, file)

            # Processar arquivos Excel
            if file.endswith(".xlsx"):
                print(f"Carregando arquivo Excel: {file_path}")
                try:
                    df = pd.read_excel(file_path, usecols=colunas_desejadas)
                    dfs.append(df)
                except Exception as e:
                    print(f"Erro ao ler {file}: {e}")
                time.sleep(10)

            # Processar arquivos CSV
            elif file.endswith(".csv"):
                print(f"Carregando arquivo CSV: {file_path}")
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        sample = f.read(2048)
                        f.seek(0)
                        try:
                            dialect = csv.Sniffer().sniff(sample)
                            sep = dialect.delimiter
                        except Exception:
                            sep = '\t'  # Fallback

                    df = pd.read_csv(file_path, sep=sep, usecols=lambda x: x in colunas_desejadas)
                    dfs.append(df)
                except Exception as e:
                    print(f"Erro ao ler {file}: {e}")
                time.sleep(10)
        
        if dfs:
            df_cadastro = pd.concat(dfs, ignore_index=True).drop_duplicates()
            print(f"Dados carregados e consolidados.")
            time.sleep(10)

            base_filename = f"Cad_Consolidado_{get_today_number()}"
            save_split_xlsx(df_cadastro, base_folder, base_filename)
            time.sleep(10)

            delete_old_files(base_folder, base_filename)
            time.sleep(10)
        else:
            print(f"Sem dados para processar.")
    
    except Exception as e:
        print(f"Erro geral no processamento: {e}")

def save_split_xlsx(df, output_folder, base_filename, max_rows=1048576):
    try:
        total_rows = df.shape[0]
        num_files = math.ceil(total_rows / max_rows)

        print(f"Total de {total_rows} linhas. Dividindo em {num_files} arquivos.")
        time.sleep(10)

        for i in range(num_files):
            start_row = i * max_rows
            end_row = min((i + 1) * max_rows, total_rows)
            output_file = os.path.join(output_folder, f"{base_filename}_Pt{i+1}.xlsx")
            df.iloc[start_row:end_row].to_excel(output_file, index=False)
            print(f"Arquivo salvo: {output_file} ({end_row - start_row} linhas)")
            time.sleep(10)
    
    except Exception as e:
        print(f"Erro ao salvar arquivos: {e}")

def delete_old_files(folder, current_filename):
    try:
        for file in os.listdir(folder):
            if file.endswith(".xlsx") and current_filename not in file:
                file_path = os.path.join(folder, file)
                print(f"Excluindo arquivo: {file_path}")
                send2trash(file_path)
                time.sleep(10)
    
    except Exception as e:
        print(f"Erro ao excluir arquivos antigos: {e}")

def get_today_number():
    return datetime.today().strftime('%Y%m%d')

# Executar
load_and_save_cadastro_data()


Processando arquivos da pasta: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\Cadastro


In [ ]:
#!/usr/bin/env python
# coding: utf-8

import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Função para enviar e-mails via Gmail
def enviar_email(remetente, senha, destinatarios, assunto, corpo):
    try:
        message = MIMEMultipart()
        message['Subject'] = assunto
        message['From'] = remetente
        message['To'] = ', '.join(destinatarios)
        
        # Adicionando o corpo do e-mail
        message.add_header('Content-Type', 'text/html')
        message.set_payload(corpo)

        # Conexão com o servidor SMTP do Gmail
        context = ssl.create_default_context()
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls(context=context)  # Habilita TLS (criptografia)
            server.login(remetente, senha)    # Login com a senha de app
            server.sendmail(
                remetente, destinatarios,
                message.as_string().encode('utf_8')
            )
        print(f"E-mail enviado com sucesso de {remetente}.")
        return True
    except Exception as e:
        print(f"Falha ao enviar e-mail com {remetente}. Erro: {e}")
        return False

# Definir destinatários
email_to = ['marcelo.carvalho@visionone.com.br', 'gabrielly.vetore@visionone.com.br']

# Corpo do e-mail (mensagem fixa)
corpo = r'''
    <p>Prezado(a),</p>
    <p>SUCESSO AO EXTRAIR CADASTRO QVIS</p>
    <p>Esta é uma mensagem enviada automaticamente. Por favor, não a responda.</p>
'''

# Assunto do e-mail
assunto = 'SUCESSO CADASTRO QVIS'

# Remetente e senha de aplicativo para Gmail
remetente_gmail = 'mcc.visionone@gmail.com'
senha_gmail = 'bqwk zcgm vgeh bcgp'  # Substitua com sua senha de aplicativo gerada

# Tentar enviar com Gmail
sucesso = enviar_email(remetente_gmail, senha_gmail, email_to, assunto, corpo)

if sucesso:
    print("E-mail enviado com sucesso!")
else:
    print("Falha ao enviar e-mail.")

In [ ]:
fim_tempo = datetime.now()
tempo = fim_tempo - inicio_tempo

# Converter para minutos e segundos
minutos, segundos = divmod(tempo.total_seconds(), 60)

print(f"Tempo de execução: {int(minutos)} minutos e {segundos:.2f} segundos")